In [2]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import datetime

In [3]:
%load_ext tensorboard

In [4]:
rm -rf ./logs/

In [5]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [6]:
train_path = '/Users/amitaflalo/Desktop/deepnav/data/train/'
test_path = '/Users/amitaflalo/Desktop/deepnav/data/test/'
interval = 50

In [7]:
def file_select_train(name):  
    global accelometer
    global gyroscope
    global speed
    global df

    df = pd.read_csv(train_path + name , header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    speed = df.iloc[:, 9:10]

In [8]:
def file_select_test(name):  
    global accelometer
    global gyroscope
    global speed
    global df

    df = pd.read_csv(test_path + name , header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    speed = df.iloc[:, 9:10]

In [9]:
def get_batch(batch_size, index):

    global accelometer
    global gyroscope
    global speed
    global df
    
    idx = index

    #input to Network[accelometer data]
    ##########################################################################################

    batch_accelometer = [accelometer.iloc[i * interval : i * interval + interval,:] for i in range(idx,idx + batch_size)]

    batch_accelometer = np.array(batch_accelometer, dtype='float64')


    #ground truth[acceleraion(m/s)]
    ########################################################################################## 

    batch_acceleraion = [speed.iloc[i * interval,:] for i in range(idx + 1,idx + batch_size + 1)]

    batch_acceleraion = np.array(batch_acceleraion,  dtype='float64')




    return batch_accelometer, batch_acceleraion

In [10]:
file_select_train('locationData.csv')

In [11]:
x,y = get_batch(50,0)

In [12]:
x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
x = tf.keras.layers.MaxPool1D(3)(x)
x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)


In [13]:
x.shape

TensorShape([50, 128])

In [14]:
def build_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation='relu', input_shape = (50,3)),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation='relu'),
    tf.keras.layers.MaxPool1D(3),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation='relu'),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation='relu'),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1)
  ])

  return model
model = build_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 48, 64)            640       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 46, 64)            12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 13, 64)            12352     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 11, 64)            12352     
_________________________________________________________________
flatten (Flatten)            (None, 704)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [15]:
# def build_model():
#   model = tf.keras.Sequential([
#     tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu', input_shape = (50,3)),
#     tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu'),
    
#     tf.keras.layers.MaxPool1D(3),
    
#     tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu'),
#     tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu'),
    
#     tf.keras.layers.Dense(128, activation='relu'),


#     tf.keras.layers.Dense(1)
#   ])

#   return model

# model = build_model()
# print(model.summary())


In [16]:
def compute_loss(labels, logits):
  mse = tf.keras.losses.MeanSquaredError()
  loss = mse(labels, logits)
  return loss

def precent_loss(labels, logits):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    loss = mae(labels, logits)
    return loss

In [17]:
# Define our metrics for tensorboard
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

In [37]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
EPOCHS = 100  # Increase this to train longer
batch_size = 10  # Experiment between 1 and 64
learning_rate = 1e-6  # Experiment between 1e-5 and 1e-1

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
      y_hat = model(x) 
      loss = compute_loss(y, y_hat)


  # Now, compute the gradients 
  grads = tape.gradient(loss, model.trainable_variables) 
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  #metrics
  train_loss(loss)

@tf.function
def test_step(x, y): 
    y_hat = model(x) 
    loss = precent_loss(y, y_hat)
    test_loss(loss)

In [39]:
##################
# Begin training!#
##################

for epoch in range(EPOCHS):

  # Grab a batch and propagate it through the network
  for filename in os.listdir(train_path):
    if not filename.startswith('.'):
      file_select_train(filename)
      for index in range(0, ((len(df) - (batch_size + 1) * interval) // interval), batch_size):
        x_batch, y_batch = get_batch(batch_size, index)
        train_step(x_batch, y_batch)

  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)

  for filename in os.listdir(test_path):
    if not filename.startswith('.'):
      file_select_test(filename)
      for index in range(0, ((len(df) - (batch_size + 1) * interval) // interval), batch_size):
        x_batch, y_batch = get_batch(batch_size, index)
        test_step(x_batch, y_batch)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss_test', test_loss.result(), step=epoch)

  # Update the model with the changed weights!
  if epoch % 10 == 0:     
    model.save_weights(checkpoint_prefix)

  template = 'epoch {}, Loss: {}, Test Loss: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         test_loss.result())) 


  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

epoch 1, Loss: 314577.34375, Test Loss: 930.5774536132812
epoch 2, Loss: 252121.625, Test Loss: 925.1773681640625
epoch 3, Loss: 252121.625, Test Loss: 921.1731567382812
epoch 4, Loss: 252121.625, Test Loss: 917.2348022460938
epoch 5, Loss: 252121.625, Test Loss: 913.5752563476562
epoch 6, Loss: 252121.625, Test Loss: 910.3853149414062
epoch 7, Loss: 252121.625, Test Loss: 907.2840576171875
epoch 8, Loss: 252121.625, Test Loss: 904.403076171875
epoch 9, Loss: 252121.625, Test Loss: 901.6700439453125
epoch 10, Loss: 252121.625, Test Loss: 899.0266723632812
epoch 11, Loss: 252121.625, Test Loss: 896.412841796875
epoch 12, Loss: 252121.625, Test Loss: 893.7931518554688
epoch 13, Loss: 252121.625, Test Loss: 891.2327270507812
epoch 14, Loss: 252121.625, Test Loss: 888.7079467773438
epoch 15, Loss: 252121.625, Test Loss: 886.1831665039062
epoch 16, Loss: 252121.625, Test Loss: 883.6942138671875
epoch 17, Loss: 252121.625, Test Loss: 881.3058471679688
epoch 18, Loss: 252121.625, Test Loss: 8

In [ ]:
x,y = get_batch(1,50)
model.predict(x)

In [60]:
y

array([[0.96201709]])

In [12]:
%tensorboard --logdir /Users/amitaflalo/Desktop/deepnav/logs


Reusing TensorBoard on port 6006 (pid 11502), started 3 days, 1:22:34 ago. (Use '!kill 11502' to kill it.)